<a href="https://colab.research.google.com/github/clear21/Answer-Mashine/blob/master/RNN_jeopardy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import urllib.request
import json
import pandas as pd
import numpy as np

from keras.layers import Dense , Activation , SimpleRNN
from keras.models import Sequential

from datetime import datetime

Using TensorFlow backend.


In [0]:
#■関数

#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f

In [2]:
#QAデータセットの取得（JEOPARDY_CSV.csv）
files.upload()

# qa_data_url = r'https://drive.google.com/file/d/0BwT5wj_P7BKXUl9tOUJWYzVvUjA/view'
# qa_file_name = 'JEOPARDY_CSV.csv'
# urllib.request.urlretrieve(qa_data_url , qa_file_name)

Saving JEOPARDY_CSV.csv to JEOPARDY_CSV.csv


In [0]:
#QAデータをデータフレーム化（Question と Answer を使用する）
qa_csv = r'JEOPARDY_CSV.csv'
qa_df = pd.read_csv(qa_csv)[[' Question' , ' Answer']]

#画像URLありの質問　＆　Answerが１wordでない　を除外
qa_df = qa_df[~qa_df[' Question'].str.contains('<a href=')]
print('画像URLありの質問' , '除外後' , len(qa_df))

qa_df[' Answer'] = qa_df[' Answer'].str.strip()
qa_df = qa_df[~qa_df[' Answer'].str.contains(' ' , na = True)]
print('Answerが１wordでない' , '除外後' , len(qa_df))
qa_df = qa_df.reset_index(drop = True)


#テストのため、行数削減
qa_df = qa_df[:2000]
qa_df = qa_df.reset_index(drop = True)

print('len(qa_df)' , len(qa_df))

In [0]:
#関数：文字列→単価リスト（to_word_set～で使用）
#第二引数：「Q」＝Question用処理、「A」＝Answer用処理
def sentence2wordlist(sentence , QorA):
    sentence = str(sentence)
    if QorA == 'Q':
        return list(map(lambda x : x.lower() , sentence.replace(',' , ' , ').replace('.' , ' . ').replace('?' , ' ? ').replace('"' , '').replace('$' , ' $ ').replace('(' , ' ').replace(')' , ' ').split(' ')))
    elif QorA == 'A':
        return sentence.lower()

#関数：文字列のリスト→単語の集合（重複無）※Question用
def to_word_set_Qustion(sentence_list):
    word_split_list = []
    for sentence in sentence_list:
        word_split_list += sentence2wordlist(sentence , 'Q')
    return set(word_split_list)

#関数：文字列のリスト→単語の集合（重複無）※Answer用
def to_word_set_Answer(sentence_list):
    word_split_list = []
    for sentence in sentence_list:    
        word_split_list.append(sentence2wordlist(sentence , 'A'))
    return set(word_split_list)
  
#単語辞書作成
#--単語一覧
q_word_set = to_word_set_Qustion(qa_df[' Question'])
a_word_set = to_word_set_Answer(qa_df[' Answer'])

# word_set = q_word_set | a_word_set
# del q_word_set , a_word_set

#Q用：word2id_q、id2word_q　　A用：word2id_a、id2word_a
#--Q用
word_list_q = sorted(list(q_word_set))
word2id_q = {word_list_q[i] : i for i in range(len(word_list_q))}
id2word_q = {i : word_list_q[i] for i in range(len(word_list_q))}

del q_word_set , word_list_q

#--A用
word_list_a = sorted(list(a_word_set))
word2id_a = {word_list_a[i] : i for i in range(len(word_list_a))}
id2word_a = {i : word_list_a[i] for i in range(len(word_list_a))}

del a_word_set , word_list_a

# word_list = sorted(list(word_set))
# word2id = {word_list[i] : i for i in range(len(word_list))}
# id2word = {i : word_list[i] for i in range(len(word_list))}
# del word_set , word_list

In [22]:
#モデルの型
HIDDEN_SIZE = 512
BATCH_SIZE = 1
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE 
                    , return_sequences = False
                    , input_shape = (None , len(word2id_q))))
model.add(Dense(len(word2id_a)))
model.add(Activation("softmax"))

model.compile(loss = "categorical_crossentropy" , optimizer = "rmsprop")

#入力値として、各Questionに対して、SEQLEN文字ずつ区切ったものを作成し、Answerをラベルとする。
one_fit_qa_num = 1
SEQLEN = 3

word_split_question_list = []
word_split_answer_list = []

start = datetime.now()

for qa_no in range(len(qa_df)):    
    word_split_question_list.append( sentence2wordlist(qa_df[' Question'][qa_no] , 'Q') )
    word_split_answer_list.append( sentence2wordlist(qa_df[' Answer'][qa_no] , 'A') )
    
    if (qa_no % (one_fit_qa_num - 1) == 0 or qa_no == len(qa_df) - 1) and qa_no != 0 :
        #入力用
        word_per_SEQLEN = []
        #ラベル
        y = []
        
        for word_split_question , word_split_answer in zip(word_split_question_list , word_split_answer_list):
            append_y_size = 0
            
            #入力
            if (len(word_split_question) - SEQLEN + 1) > 0:
                for i in range(len(word_split_question) - SEQLEN + 1):
                    word_per_SEQLEN.append(word_split_question[i : i + SEQLEN])
                    append_y_size += 1
            else :
                word_per_SEQLEN.append(word_split_question + ['' for i in range(SEQLEN - len(word_split_question))])
                append_y_size += 1
            
            #ラベル
            y = y + [[1 if j == word2id_a[word_split_answer] else 0 for j in range(len(word2id_a))] for i in range(append_y_size)]
        
        X = np.zeros((len(word_per_SEQLEN) , SEQLEN , len(word2id_q)) , dtype = np.bool)
        for i in range(len(word_per_SEQLEN)):
            w_list = word_per_SEQLEN[i]
            for j in range(SEQLEN):
                wid = word2id_q[w_list[j]]
                X[i , j , wid] = 1

        #×　ラベルの値
        #×　y = [[1 if j == word2id[word_split_answer] else 0 for j in range(len(word2id))] for i in range(len(word_per_SEQLEN))]  
        
        #学習
        model.fit(np.array(X) , np.array(y) , batch_size = BATCH_SIZE , epochs = NUM_EPOCHS_PER_ITERATION)
        
        #1学習の時間
        end = datetime.now()
        print('learn time:' , end - start)
        
        #初期化
        word_split_question_list = []
        word_split_answer_list = []
        
        start = datetime.now()
        
        #if qa_no == (one_fit_qa_num - 1):
            #print('X' , np.array(X).shape)
            #print('y' , len(y))
            #break

Epoch 1/4
2596/2596 [==============================] - 38s 15ms/step - loss: 5.9528
Epoch 2/4
2596/2596 [==============================] - 33s 13ms/step - loss: 5.1513
Epoch 3/4
2596/2596 [==============================] - 34s 13ms/step - loss: 4.6276
Epoch 4/4
2596/2596 [==============================] - 34s 13ms/step - loss: 4.2840
learn time: 0:02:21.396222
Epoch 1/4
2530/2530 [==============================] - 34s 13ms/step - loss: 6.0026
Epoch 2/4
2530/2530 [==============================] - 33s 13ms/step - loss: 5.2011
Epoch 3/4
2530/2530 [==============================] - 33s 13ms/step - loss: 4.7903
Epoch 4/4
2530/2530 [==============================] - 34s 13ms/step - loss: 4.4392
learn time: 0:02:14.694067
Epoch 1/4
2539/2539 [==============================] - 31s 12ms/step - loss: 6.2734
Epoch 2/4
2539/2539 [==============================] - 33s 13ms/step - loss: 5.3280
Epoch 3/4
2539/2539 [==============================] - 38s 15ms/step - loss: 4.8648
Epoch 4/4
2539/2539 [=

In [0]:
#■関数
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list) , len(word2id_q)) , dtype = np.bool)
    for j in range(len(wid_list)):
        return_X[0 , j , wid_list[j]] = 1
    return return_X

test_word_list = ['last' , '8' , 'years' , 'of' , 'his']
#test_word_list = ['his' , 'life' , 'galileo' , 'was' , 'under']
#test_word_list = ['galileo' , 'galileo' , 'galileo' , 'galileo' , 'galileo']
#test_word_list = ['the' , '13th' , 'amendment' , ',' , 'which' , 'abolished' , 'slavery' , ',' , 'was' , 'ratified' , 'in' , 'this' , 'year']
#test_word_list = ['in' , '1990' , 'this' , 'country' , 'named' , 'a' , 'new' , 'capital' , 'city']
test_X = create_X(test_word_list)

pred = model.predict(test_X , verbose = 0)[0]

In [54]:
out_top_n = 10

enum_pred = list(enumerate(pred))
enum_pred.sort(key = lambda x : -x[1])

top_enum_pred = enum_pred[:out_top_n]
for i in range(len(top_enum_pred)):
    print((i+1) , '位' , id2word_a[top_enum_pred[i][0]] , top_enum_pred[i][1] )

print(id2word_a[np.argmax(pred)])

1 位 monaco 0.29956326
2 位 watermelon 0.19711296
3 位 gestalt 0.14072312
4 位 superbad 0.1083664
5 位 amway 0.096104786
6 位 ham 0.065872125
7 位 platinum 0.03211968
8 位 london 0.02191893
9 位 zirconium 0.019370265
10 位 dartmouth 0.0015733135
monaco


In [39]:
out_top_n = 10

enum_pred = list(enumerate(pred))
enum_pred.sort(key = lambda x : -x[1])

top_enum_pred = enum_pred[:out_top_n]
for i in range(len(top_enum_pred)):
    print((i+1) , '位' , id2word_a[top_enum_pred[i][0]] , top_enum_pred[i][1] )

print(id2word_a[np.argmax(pred)])

1 位 germany 0.08067987
2 位 alaska 0.056034524
3 位 japan 0.036893297
4 位 florida 0.03198273
5 位 france 0.024513124
6 位 china 0.024512284
7 位 india 0.024025247
8 位 washington 0.022348082
9 位 egypt 0.017369218
10 位 scotland 0.015362644
germany


In [50]:
id2word_a

{0: '"aida"',
 1: '"candyman"',
 2: '"carmen"',
 3: '"crazy"',
 4: '"dune"',
 5: '"electricity"',
 6: '"fallin\'"',
 7: '"goose"',
 8: '"greensleeves"',
 9: '"honey"',
 10: '"jabberwocky"',
 11: '"layla"',
 12: '"mandy"',
 13: '"moondance"',
 14: '"nevermore!"',
 15: '"pagliacci"',
 16: '"rigoletto"',
 17: '"rope-a"',
 18: '"scream"',
 19: '-1',
 20: '1',
 21: '1-900',
 22: '100%',
 23: '11',
 24: '12',
 25: '14',
 26: '1865',
 27: '1930s',
 28: '1941',
 29: '1944',
 30: '1960',
 31: '1970s',
 32: '1972',
 33: '1984',
 34: '19th',
 35: '2,070',
 36: '20',
 37: '21',
 38: '24',
 39: '26',
 40: '3',
 41: '35mm',
 42: '360',
 43: '4',
 44: '48',
 45: '5',
 46: '6',
 47: '750',
 48: '8',
 49: '84',
 50: '911',
 51: '98',
 52: 'aaron',
 53: 'abscam',
 54: 'academia',
 55: 'acadia',
 56: 'acapulco',
 57: 'accommodations',
 58: 'accordion',
 59: 'achievement',
 60: 'acquired',
 61: 'acura',
 62: 'adenoids',
 63: 'adieu',
 64: 'adonis',
 65: 'aerosmith',
 66: 'aeschylus',
 67: 'aesop',
 68: 'a

In [30]:
#Answer が 1word　でない　を除外する前
print(len(word2id))

9312


In [37]:
#Answer が 1word　でない　を除外した後
print(len(word2id))

8822


In [40]:
print(word2id['last'])
print(word2id['8'])
print(word2id['years'])
print(word2id['of'])
print(word2id['his'])


print(word2id['galileo'])
print(word2id['was'])


4933
604
9254
5997
4220
3724
8958


In [28]:
qa_df

,Question,Answer
0,"For the last 8 years of his life, Galileo was ...",Copernicus
1,The city of Yuma in this state has a record av...,Arizona
2,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
3,"In the winter of 1971-72, a record 1,122 inche...",Washington
4,"This company's Accutron watch, introduced in 1...",Bulova
5,"A small demon, or a mischievous child (who mig...",imp
6,Africa's lowest temperature was 11 degrees bel...,Morocco
7,"In geologic time one of these, shorter than an...",era
8,"The Kirschner brothers, Don & Bill, named this...",K2
9,"A single layer of paper, or to perform one's c...",ply


In [8]:
a_word_set
#word2id

{'(omar) bradley',
 'joseph smith',
 'brussels',
 'black death',
 'jakarta',
 'the new world',
 'medellin',
 'pods',
 'jim carrey',
 'ordinary people',
 'california',
 'drooling',
 'soft',
 'walt disney',
 'milkweed',
 'the nobel prizes',
 'why does swiss cheese have holes?',
 'the emerald city',
 '750',
 'tnt',
 'alexander graham bell',
 'arms',
 'bunker',
 'old mother hubbard',
 'rhizomes',
 '\\"death on the nile\\"',
 'easter island',
 '"rope-a"',
 'the indy 500',
 'harold stratton',
 'steadicam',
 'tooth',
 'coldplay',
 'prologue',
 'oh! calcutta!',
 'carl sandburg',
 'tiger woods',
 'gloves',
 'tee time',
 'urban outfitters',
 'psycho',
 "the (runnin') rebels",
 'maids',
 'kalahari',
 'abraham lincoln',
 'the canterbury tales',
 'zoroastrianism',
 'laugh track',
 'lake champlain',
 'hart to hart',
 'chris rock',
 'why does the snake crawl on the ground?',
 'prince william',
 'john cheever',
 '(frank) gehry',
 'august wilson',
 '"the village blacksmith"',
 'helen',
 '(art) garfunke